In [1]:
from datetime import datetime
from sklearn.utils import compute_class_weight
from tensorflow import keras
from src.data.load_data import train_X, train_y, validation_X, validation_y, test_X, test_y, df_ratings
from src.models.metrics import sps
from src.models.model import NextItemPredictor
from src.models.predict import predict_10
from src.models.train import train
import numpy as np

/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/src/data/load_data.py:27: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  names=['user_id', 'movie_id', 'rating', 'timestamp'])


In [2]:
%load_ext tensorboard
logdir = "../logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
%tensorboard --logdir ../logs/

Launching TensorBoard...

In [3]:
model = NextItemPredictor()
model.compile()

In [4]:
class_weight = compute_class_weight('balanced', np.unique(train_y), train_y)
train(
    model,
    train_X=train_X,
    train_y=train_y,
    validation_X=validation_X,
    validation_y=validation_y,
    class_weight=class_weight,
    batch_size=32,
    epochs=1,
    callbacks=[tensorboard_callback],
    description='batch100-epochs30'
)

Train on 1966 samples, validate on 858 samples
1966/1966 [==============================] - 8s 4ms/sample - loss: 8.0431 - val_loss: 7.8571


In [5]:
y_pred = predict_10(model, test_X)

In [8]:
sps_score = sps(test_y, y_pred)
f = open("../logs/metrics_logs.txt", "a")
f.write(str(datetime.now()) + '\n')
f.write('sps score: {} \n'.format(sps_score))
f.close()

In [ ]:
# TODO: bidirectional, recommend 10 items, try to split by users?

# attention, hierarchy